In [1]:
from importlib import reload
import sys

src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

max_length = 128

In [2]:
import data_generator;reload(data_generator)

train_data, val_data, test_data = data_generator.GetData(max_length, sample=1000)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
100%|██████████| 3000/3000 [00:00<00:00, 3382.13it/s]


        tag  cat  occurences
0    B-MISC    0           5
1     I-LOC    1        2560
2    I-MISC    2        1023
3     I-ORG    3        1774
4     I-PER    4        8135
5         O    5       52468
6  [nerCLS]    6        3000
7  [nerPAD]    7      295967
8  [nerSEP]    8        3000
9    [nerX]    9       16068

                tag  cat  occurences
0  AFRICAN-AMERICAN    0       43110
1          EUROPEAN    1       38923
2         [raceCLS]    2        3000
3         [racePAD]    3      295967
4         [raceSEP]    4        3000

           tag  cat  occurences
0       FEMALE    0       40783
1         MALE    1       41250
2  [genderCLS]    2        3000
3  [genderPAD]    3      295967
4  [genderSEP]    4        3000



In [3]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import model_utils; reload(model_utils)
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = model_utils.NER(max_length)
    
model.generate(bert_train_layers=4)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, None, 768)    108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [ ]:
train = True
debias = False

if train:

    model.fit(
        sess,
        train_data,
        val_data,
        epochs=8,
        batch_size=32,
        debias=debias
    ) 
    
else: 
    model_utils.initialize_vars(sess)


acc_ner: 0.87; acc_ner_non_other: 0.36;  acc_gender: 0.50; acc_race: 0.53


In [ ]:
bias = model.getBiasedPValues(test_data, num_iterations=10000)

In [ ]:
bias

In [ ]:
sess.close()